In [123]:
# Imports
from cmath import e
import cv2
import numpy as np
import os
import pandas as pd
# Imports
from cmath import e
from time import time
import cv2
import numpy as np
from skimage.feature import graycomatrix, graycoprops
from skimage.measure import shannon_entropy as Entropy
from sklearn import preprocessing, svm
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score
from sklearn.svm import SVC
from tuning import svmTuner
import utils
import os
import pandas as pd
import numpy as np

### TRY WITH ONE IMG:

In [124]:
# Loading the image
img = cv2.imread("HOG.jpg").astype('uint8')
print(img.shape)
#REF https://learnopencv.com/handwritten-digits-classification-an-opencv-c-python-tutorial/
#  In most cases we will use the default value of these parameters:
#  derivAperture, winSigma, histogramNormType, L2HysThreshold, gammaCorrection and nlevels 
derivAperture = 1
winSigma = -1.
histogramNormType = 0
L2HysThreshold = 0.2
gammaCorrection = 1
nlevels = 64
#-----------------------------------------------------
#REF https://www.intel.com/content/www/us/en/develop/documentation/ipp-dev-reference/top/volume-2-image-processing/computer-vision/feature-detection-functions/histogram-of-oriented-gradients-hog-descriptor.html
# According to [Dalal05] the recommended values for the HOG parameters are:
# 1D centered derivative mask [-1, 0, +1]
# Detection window size is 64x128
# Cell size is 8x8
# Block size is 16x16 (2x2 cells)
#-----------------------------------------------------
img=cv2.resize(img,(64,128))
winSize = (64, 128)
# The notion of blocks exist to tackle illumination variation.
# A large block size makes local changes less significant while a smaller block 
# size weights local changes more.
# Typically blockSize is set to 2 x cellSize
blockSize = (16, 16)
# The blockStride determines the overlap between neighboring blocks
# and controls the degree of contrast normalization.
# Typically a blockStride is set to 50% of blockSize.
blockStride = (8, 8)
#The cellSize is chosen based on the scale of the features important to do the classification.
# A very small cellSize would blow up the size of the feature vector
#  and a very large one may not capture relevant information
cellSize = (8,8)
# nbins sets the number of bins in the histogram of gradients.
# The authors of the HOG paper had recommended a 
# value of 9 to capture gradients between 0 and 180 degrees in 20 degrees increments
nbins = 9
# Typically gradients can have any orientation between 0 and 360 degrees. 
# These gradients are referred to as “signed” gradients as opposed to “unsigned” 
# gradients that drop the sign and take values between 0 and 180 degrees. 
# In the original HOG paper, unsigned gradients were used for pedestrian detection. 
# In my experiments,for this problem, signed gradients produced slightly better results.
signedGradient = True
#-----------------------------------------------------
hog = cv2.HOGDescriptor(winSize,blockSize,blockStride,
cellSize,nbins,derivAperture,winSigma,histogramNormType
,L2HysThreshold,gammaCorrection,nlevels, signedGradient)
# img = cv2.resize(img, (64,128))

# img	Matrix of the type CV_8U containing an image where 
# HOG features will be calculated descriptors	Matrix of the type CV_32F
#----------------------------------------------------------
# CV_8U is unsigned 8bit/pixel - ie a pixel can have values 0-255, 
# this is the normal range for most image and video formats
h = hog.compute(img)
print(h)

(375, 500, 3)
[0. 0. 0. ... 0. 0. 0.]


## HOG STEPS:

#REF https://www.intel.com/content/www/us/en/develop/documentation/ipp-dev-reference/top/volume-2-image-processing/computer-vision/feature-detection-functions/histogram-of-oriented-gradients-hog-descriptor.html
* Implementation of the HOG descriptor algorithm is as follows:
1. Divide the image into small connected regions called cells, and for each cell compute a histogram of gradient directions or edge orientations for the pixels within the cell.
2. Discretize each cell into angular bins according to the gradient orientation.
3. Each cell's pixel contributes weighted gradient to its corresponding angular bin.
4. Groups of adjacent cells are considered as spatial regions called blocks. The grouping of cells into a block is the basis for grouping and normalization of histograms.
5. Normalized group of histograms represents the block histogram. The set of these block histograms represents the descriptor.

## HOG:

In [125]:
def calc_HOG(filename, outputFileName):
    # Loading the image
    img = cv2.imread(filename).astype('uint8')
    #REF https://learnopencv.com/handwritten-digits-classification-an-opencv-c-python-tutorial/
    #  In most cases we will use the default value of these parameters:
    #  derivAperture, winSigma, histogramNormType, L2HysThreshold, gammaCorrection and nlevels 
    derivAperture = 1
    winSigma = -1.
    histogramNormType = 0
    L2HysThreshold = 0.2
    gammaCorrection = 1
    nlevels = 64
    #-----------------------------------------------------
    #REF https://www.intel.com/content/www/us/en/develop/documentation/ipp-dev-reference/top/volume-2-image-processing/computer-vision/feature-detection-functions/histogram-of-oriented-gradients-hog-descriptor.html
    # According to [Dalal05] the recommended values for the HOG parameters are:
    # 1D centered derivative mask [-1, 0, +1]
    # Detection window size is 64x128
    # Cell size is 8x8
    # Block size is 16x16 (2x2 cells)
    #-----------------------------------------------------
    img=cv2.resize(img,(64,128))
    winSize = (64, 128)
    # The notion of blocks exist to tackle illumination variation.
    # A large block size makes local changes less significant while a smaller block 
    # size weights local changes more.
    # Typically blockSize is set to 2 x cellSize
    blockSize = (8, 8)
    # The blockStride determines the overlap between neighboring blocks
    # and controls the degree of contrast normalization.
    # Typically a blockStride is set to 50% of blockSize.
    blockStride = (4, 4)
    #The cellSize is chosen based on the scale of the features important to do the classification.
    # A very small cellSize would blow up the size of the feature vector
    #  and a very large one may not capture relevant information
    cellSize = (4,4)
    # nbins sets the number of bins in the histogram of gradients.
    # The authors of the HOG paper had recommended a 
    # value of 9 to capture gradients between 0 and 180 degrees in 20 degrees increments
    nbins = 9
    # Typically gradients can have any orientation between 0 and 360 degrees. 
    # These gradients are referred to as “signed” gradients as opposed to “unsigned” 
    # gradients that drop the sign and take values between 0 and 180 degrees. 
    # In the original HOG paper, unsigned gradients were used for pedestrian detection. 
    # In my experiments,for this problem, signed gradients produced slightly better results.
    signedGradient = True
    #-----------------------------------------------------
    hog = cv2.HOGDescriptor(winSize,blockSize,blockStride,
    cellSize,nbins,derivAperture,winSigma,histogramNormType
    ,L2HysThreshold,gammaCorrection,nlevels, signedGradient)
    # img = cv2.resize(img, (64,128))

    # img	Matrix of the type CV_8U containing an image where 
    # HOG features will be calculated descriptors	Matrix of the type CV_32F
    #----------------------------------------------------------
    # CV_8U is unsigned 8bit/pixel - ie a pixel can have values 0-255, 
    # this is the normal range for most image and video formats
    descriptor = hog.compute(img)
    with open(outputFileName+'.csv', 'a') as csvfile:
        np.savetxt(csvfile, [descriptor], fmt='%f', delimiter=',')
        csvfile.close()
    return descriptor


In [126]:
def removeFileIfExists(fileName):
    if os.path.isfile(fileName):
        os.remove(fileName)

In [127]:
def writeFeaturesToFile(features, fileName):
    with open(fileName, 'a') as csvfile:
        np.savetxt(csvfile, features, fmt='%f', delimiter=',')
        csvfile.close()

In [128]:
def readFeaturesFromFile(fileName):
    CSVData = open(fileName)
    features = np.genfromtxt(CSVData, delimiter=",")
    return features

In [129]:
def extractFeaturesFromFolder(folder,outputFileName,gender):
    train_classes=[]
    features=[]
    for filename in os.listdir(folder):
        try:
            features.append(calc_HOG(folder+filename,outputFileName))
            train_classes.append(gender)
        except Exception as e:
            print(e)
            continue
    return np.array(features),np.array(train_classes)


In [130]:
def extractICDARFeatures():
    features=[]
    # read csv file
    df = pd.read_csv('train_answers.csv')
    # get the labels
    icdar_classes = df['male'].values
    print(icdar_classes.shape)
    icdar_classes_train = np.array([])
    i = 0
    for filename in os.listdir('images_gender/images/train'):
        try:
            features.append(calc_HOG('images_gender/images/train/'+filename,'icdar_hog'))
            icdar_classes_train = np.append(icdar_classes_train, icdar_classes[i//2])
            i = i + 1
        except Exception as e:
            print(e)
            continue
    icdar_classes  = icdar_classes_train
    return np.array(features),np.array(icdar_classes)

In [131]:
def FeaturesFromScratch():
    removeFileIfExists('female_hog.csv')
    removeFileIfExists('male_hog.csv')
    removeFileIfExists('icdar_hog.csv')

    # writeHeadersOfCSVFile('female.csv')
    # writeHeadersOfCSVFile('male.csv')
    # writeHeadersOfCSVFile('icdar.csv')

    f_features,f_classes = extractFeaturesFromFolder('Females/Females/','female_hog',0)
    f_features =  f_features.reshape(f_features.shape[0], -1)
    print(f_features.shape)
    print(f_classes.shape)
    m_features,m_classes = extractFeaturesFromFolder('Males/Males/','male_hog',1)
    m_features =  m_features.reshape(m_features.shape[0], -1)
    print(m_features.shape)
    print(m_classes.shape)
    i_features,i_classes = extractICDARFeatures()
    i_features = i_features.reshape(i_features.shape[0], -1)
    print(i_features.shape)
    print(i_classes.shape)
    X_train = np.concatenate((f_features,m_features,i_features ),axis=0)
    Y_train = np.concatenate((f_classes,m_classes,i_classes),axis=0)
    # Y_train = np.concatenate((train_classes,icdar_classes),axis=0)



    X_train, X_test, Y_train, Y_test = train_test_split(
        X_train, Y_train, test_size=0.2,random_state=1)

    # # getBestParamsForANN(X_train,Y_train,X_test,Y_test)
    # # getBestParamsForSVM(X_train,Y_train,X_test,Y_test)
    trainAndPredict(X_train,Y_train,X_test,Y_test)

In [132]:
def trainAndPredict(X_train,Y_train,X_test,Y_test):
    # train the classifier and predict the test data
    scaler = preprocessing.StandardScaler().fit(X_train)

    print("Training the classifier...")          
    # clf = MLPClassifier() 
    clf = SVC(C=5000.0, gamma=0.5,kernel='rbf',random_state=1,probability=True,class_weight='balanced')

    clf.fit(scaler.transform( X_train), Y_train) 
    
    print("Predicting the test data...")
    score_training = clf.score(scaler.transform( X_train), Y_train) 
    score = clf.score(scaler.transform(X_test), Y_test)
    print("Accuracy on test set: {:.4f}".format(score))
    print("Accuracy on training set: {:.4f}".format(score_training))

In [133]:
FeaturesFromScratch()

OpenCV(4.5.5) :-1: error: (-5:Bad argument) in function 'HOGDescriptor'
> Overload resolution failed:
>  - Can't parse '_blockSize'. Sequence item with index 0 has a wrong type
>  - HOGDescriptor() takes at most 1 argument (12 given)

OpenCV(4.5.5) :-1: error: (-5:Bad argument) in function 'HOGDescriptor'
> Overload resolution failed:
>  - Can't parse '_blockSize'. Sequence item with index 0 has a wrong type
>  - HOGDescriptor() takes at most 1 argument (12 given)

OpenCV(4.5.5) :-1: error: (-5:Bad argument) in function 'HOGDescriptor'
> Overload resolution failed:
>  - Can't parse '_blockSize'. Sequence item with index 0 has a wrong type
>  - HOGDescriptor() takes at most 1 argument (12 given)

OpenCV(4.5.5) :-1: error: (-5:Bad argument) in function 'HOGDescriptor'
> Overload resolution failed:
>  - Can't parse '_blockSize'. Sequence item with index 0 has a wrong type
>  - HOGDescriptor() takes at most 1 argument (12 given)

OpenCV(4.5.5) :-1: error: (-5:Bad argument) in function 'HOG

ValueError: cannot reshape array of size 0 into shape (0,newaxis)

In [ ]:
# f_features = readFeaturesFromFile('female.csv')
# m_features = readFeaturesFromFile('male.csv')
# i_features = readFeaturesFromFile('icdar.csv')

train_classes = []
# read csv file
df = pd.read_csv('./train_answers.csv')
# get the labels
icdar_classes = df['male'].values
print(icdar_classes.shape)
icdar_classes_train = np.array([])

for i in range(1, 132):
    try:
        train_classes.append(0)
    except Exception as e:
        print(e)
        continue

for i in range(1, 233):
    try:
        train_classes.append(1)
    except Exception as e:
        print(e)
        continue

for i in range(0, 564):
    try:
        icdar_classes_train = np.append(icdar_classes_train, icdar_classes[i//2])
        i = i + 1
    except Exception as e:
        print(e)
        continue

icdar_classes = icdar_classes_train

# train_classes = np.array(train_classes)





X_train = np.concatenate((f_features,m_features,i_features ),axis=0)
# Y_train = np.concatenate((f_classes,m_classes,i_classes),axis=0)
Y_train = np.concatenate((train_classes,icdar_classes),axis=0)




trainAndPredict(X_train,Y_train)



(282,)


TypeError: trainAndPredict() missing 2 required positional arguments: 'X_test' and 'Y_test'